In [ ]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 3.8MB 38.3MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 512kB 57.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=94f39b932bdff96a4030a34aa17ba5bd2de1a34382cbed33c141c690e63a4732
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [ ]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [ ]:
%reset

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

## Raw Data Loading
df = pd.read_csv('/content/drive/My Drive/MachineLearning/train.csv')
display(df.head())

## 결측치와 이상치는 없다.

## Data Split : Train data와 Test data 분리
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', axis=1, inplace=False),
                 df['label'],
                 test_size=0.3,
                 random_state=0)

# t data는 one hot encoding 할 것이기 때문에 정규화 x
# x_data는 pxel 값이기 때문에 0~255 사이의 값. 정규화 필요
## Min-Max Normaliztion
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train, x_data_test

#### Tensorflow 1.15 implementaion ### 

## one-hot encoding
sess = tf.Session()

# 몇개의 one hot category로 나눌 지 depth로 표기
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=10))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=10))

## Placeholder
# 입력되는 2차원 데이터 컬럼 784개
X = tf.placeholder(shape=[None, 784], dtype=tf.float32)
# 입력되는 정답 - 카테고리 10개
T = tf.placeholder(shape=[None, 10], dtype=tf.float32)
drop_rate = tf.placeholder(dtype=tf.float32)

#### Convolution

## 입력 데이터 형태부터 설정

# 3차원 (가로, 세로, depth) 여러 장 => 4차원
# Convolution은 4차원 데이터가 입력되어야 함 => reshape (numpy 말고 tensor의 shape 바꿈)
# column 784개 지만 원래 세로 가로 28, 흑백이라 channel, 몇개일지 모르니까 -1

x_img = tf.reshape(X,[-1, 28, 28, 1])     # (이미지 개수, height, width, channel)

### convolution layer 1
## 값이 변하니까 Variable, 일단 random으로 설정
# filter 3 x 3 첫번째 필터 채널은 이미지의 채널과 같아야 한다. 
# 이러한 filter를 cl에서 여러개 만들 수 있다. -> 32개
# 3 by 3 씩 stride 움직임
W1 = tf.Variable(tf.random.normal([3, 3, 1, 32]))  # (filter height, 
                                                   # filter width, 
                                                   # filter channel, 
                                                   # filter 개수)

# filter 하나 당 feature map 한개 => 32개의 feature map이 나옴
# size 줄이지 않고 zero padding 적용 -> padding='SAME'
L1 = tf.nn.conv2d(x_img,W1, strides=[1,1,1,1], padding='SAME') ## convolution 작업수행
L1 = tf.nn.relu(L1)  ## 이 작업의 결과 => activatio map (None, 28, 28, 32) 
# 28 28 1채널 이미지가 -> channel이 32개로, filter 개수 만큼 늘어남
# convolution 작업 수행 결과값에 relu 함수까지 적용한 것이 convolution layer의 결과물

### pooling layer 1 - 데이터 줄이기
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # 2 by 2, pooling도 strides가 있고 커널과 동일 사이즈로 잡는다.
print('L1의 shape: {}'.format(L1.shape))
# L1의 shape: (?, 14, 14, 32)
# zero padding 줘도 14로 줄어든다.


### convolution layer 2
W2 = tf.Variable(tf.random.normal([3, 3, 32, 64])) # (filter height, 
                                                   # filter width, 
                                                   # filter channel, 
                                                   # filter 개수)
L2 = tf.nn.conv2d(L1,W2, strides=[1,1,1,1], padding='SAME') ## convolution 작업수행
L2 = tf.nn.relu(L2)  ## 이 작업의 결과 => activatio map (None, 28, 28, 32) 

### pooling layer 2 - 데이터 줄이기
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# 2 by 2, pooling도 strides가 있고 커널과 동일 사이즈로 잡는다.
print('L2의 shape: {}'.format(L2.shape))
# L2의 shape: (?, 7, 7, 64)
# convolution의 결과, 이미지 특징 추출한 4차원 데이터 확보

#### FC에 넣어서 학습을 진행한다.
# => FC layer에 넣기 위해 데이터를 Flatten 처리 (1차원으로 만든다.)

### Input Layer
L2 = tf.reshape(L2, [-1,7*7*64])

## Hidden Layer - 보통 1개
# Weight & bias
# 초기값 W : He's 초기법
W3 = tf.get_variable('weight3', shape=[7*7*64,256],
                     initializer=tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.random.normal([256]))

# vanishing gradient 문제 해결을 위해, sigmoid 대신 relu 함수 사용
_layer3 = tf.nn.relu(tf.matmul(L2, W3) + b3) # 오버피팅 발생
layer3 = tf.nn.dropout(_layer3, rate=drop_rate)

W4 = tf.get_variable('weight4', shape=[256,10],
                     initializer=tf.contrib.layers.variance_scaling_initializer())
b4 = tf.Variable(tf.random.normal([10]))

# Hypothesis
logit = tf.matmul(layer3, W4) + b4
H = tf.nn.softmax(logit)

# loss 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=T))
                                                                 
# train
train = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

# parameter
num_of_epoch = 200
batch_size = 100

### 학습
def run_train(sess, train_x, train_t):
    print('### Starting Training ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / batch_size)

        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X:batch_x, 
                                                            T:batch_t, 
                                                            drop_rate:0.4})

        if step % 20 == 0:
            print('Loss : {}'.format(loss_val))
    print('### End Training ###')

    
# Accuracy
predict = tf.argmax(H,1)

# sklearn의 classification_report를 이용한 성능평가
run_train(sess, x_data_train_norm, t_data_test_onehot)
target_names = ['num 0','num 1','num 2','num 3','num 4','num 5','num 6','num 7', 'num 8','num 9']
print(classification_report(t_data_test,
                            sess.run(predict,
                                     feed_dict={X:x_data_test_norm,
                                                drop_rate:0}),
                            target_names=target_names))

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


L1의 shape: (?, 14, 14, 32)
L2의 shape: (?, 7, 7, 64)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

### Starting Training ###


InvalidArgumentError: ignored

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
